# Quantum Machine Learning Application to Classification of New Physics Signal

## Quantum Neural Networks

### Preparation of training data

In [1]:
# Tested with python 3.10.11, qiskit 0.42.1, numpy 1.23.5, scipy 1.9.3
import numpy as np
import h5py
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from collections import OrderedDict

from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import TwoLocal, ZFeatureMap, ZZFeatureMap
from qiskit.primitives import Estimator, Sampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient
from qiskit_algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit_optimization.applications import OptimizationApplication
from qiskit_ibm_runtime import Session, Sampler as RuntimeSampler
from qiskit_ibm_runtime.accounts import AccountNotFoundError
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel

ImportError: cannot import name 'QuantumCircuit' from 'qiskit' (unknown location)

In [ ]:
# Read data from file
df = pd.read_csv("data/SUSY_1K.csv",
                 names=('isSignal','lep1_pt','lep1_eta','lep1_phi','lep2_pt','lep2_eta',
                        'lep2_phi','miss_ene','miss_phi','MET_rel','axial_MET','M_R','M_TR_2',
                        'R','MT2','S_R','M_Delta_R','dPhi_r_b','cos_theta_r1'))

# Number of input features for training
feature_dim = 3

# Feature variables
if feature_dim == 3:
    selected_features = ['lep1_pt', 'lep2_pt', 'miss_ene']
elif feature_dim == 5:
    selected_features = ['lep1_pt','lep2_pt','miss_ene','M_TR_2','M_Delta_R']
elif feature_dim == 7:
    selected_features = ['lep1_pt','lep1_eta','lep2_pt','lep2_eta','miss_ene','M_TR_2','M_Delta_R']

# Number of events used in the training and testing
train_size = 20
test_size = 20

df_sig = df.loc[df.isSignal==1, selected_features]
df_bkg = df.loc[df.isSignal==0, selected_features]

# Extract the samples
df_sig_train = df_sig.values[:train_size]
df_bkg_train = df_bkg.values[:train_size]
df_sig_test = df_sig.values[train_size:train_size + test_size]
df_bkg_test = df_bkg.values[train_size:train_size + test_size]
# The first train_size events contain SUSY signal and the last train_size events do not.
train_data = np.concatenate([df_sig_train, df_bkg_train])
# The first test_size events contain SUSY signal and the last test_size events do not.
test_data = np.concatenate([df_sig_test, df_bkg_test])

# Label
train_label = np.zeros(train_size * 2, dtype=int)
train_label[:train_size] = 1
test_label = np.zeros(train_size * 2, dtype=int)
test_label[:test_size] = 1

train_label_one_hot = np.zeros((train_size * 2, 2))
train_label_one_hot[:train_size, 0] = 1
train_label_one_hot[train_size:, 1] = 1
test_label_one_hot = np.zeros((test_size * 2, 2))
test_label_one_hot[:test_size, 0] = 1
test_label_one_hot[test_size:, 1] = 1

#datapoints, class_to_label = split_dataset_to_data_and_labels(test_input)
#datapoints_tr, class_to_label_tr = split_dataset_to_data_and_labels(training_input)

mms = MinMaxScaler((-1, 1))
norm_train_data = mms.fit_transform(train_data)
norm_test_data = mms.transform(test_data)

### State Preparation with Feature Map

In [2]:
#feature_map = ZFeatureMap(feature_dimension=feature_dim, reps=1)
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=1, entanglement='circular')
feature_map.decompose().draw('mpl')

### State Transformation with Variational Form

In [ ]:
ansatz = TwoLocal(num_qubits=feature_dim, rotation_blocks=['ry', 'rz'], entanglement_blocks='cz', entanglement='circular', reps=3)
#ansatz = TwoLocal(num_qubits=feature_dim, rotation_blocks=['ry'], entanglement_blocks='cz', entanglement='circular', reps=3)
ansatz.decompose().draw('mpl')

### Measurement and Model Output

In [ ]:
# Use Sampler instead of backend
sampler = Sampler()

# When using quantum hardware
# instance = 'ibm-q/open/main'

# try:
#     service = QiskitRuntimeService(channel='ibm_quantum', instance=instance)
# except AccountNotFoundError:
#     service = QiskitRuntimeService(channel='ibm_quantum', token='__paste_your_token_here__',
#                                    instance=instance)

# backend_name = 'ibm_washington'
# session = Session(service=service, backend=backend_name)

# sampler = RuntimeSampler(session=session)

maxiter = 300

optimizer = COBYLA(maxiter=maxiter, disp=True)

objective_func_vals = []
# Draw the value of objective function every time when the fit() method is called
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    #print('obj_func_eval =',obj_func_eval)

    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(objective_func_vals)
    plt.show()

vqc = VQC(num_qubits=feature_dim,
          feature_map=feature_map,
          ansatz=ansatz,
          loss="cross_entropy",
          optimizer=optimizer,
          callback=callback_graph,
          sampler=sampler)

### Execute with Simulator

In [ ]:
vqc.fit(norm_train_data, train_label_one_hot)

train_score = vqc.score(norm_train_data, train_label_one_hot)
test_score = vqc.score(norm_test_data, test_label_one_hot)

print(f'--- Classification Train score: {train_score} ---')
print(f'--- Classification Test score:  {test_score} ---')

## Quantum Kernel Method

Let us first implement a quantum circuit to encode input data as QuantumCircuit instance. You could use, e.g, ZFeatureMap or ZZFeatureMap used abovem or write the circuit by hand by creating an empty QuantumCircuit object and using Parameter or ParameterVector.

You can change the number of qubits used, but it appears that FidelityQuantumKernel class that we use later will work better if the number of input features is the same as the number of qubits.

In [3]:
##################
### EDIT BELOW ###
##################

#回路をスクラッチから書く場合
input_features = ParameterVector('x', feature_dim)
num_qubits = feature_dim
feature_map = QuantumCircuit(num_qubits)
# ...

##################
### EDIT ABOVE ###
##################

Next step is to create a quantum circuit named "manual_kernel", by which the kernel matrix is calculated using the feature map defined above. Qiskit has an API (FidelityQuantumKernel class) that does this automatically and we will use that next. Here you could consider constructing the circuit by starting with an empty QuantumCircuit object and placing the feature map defined above.

In [4]:
manual_kernel = QuantumCircuit(feature_map.num_qubits)

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

manual_kernel.measure_all()

Execute the circuit with simulator and calculate the probability of measuring 0 in all qubits, $|\langle0^{\otimes n}|U_{\text{in}}^\dagger(x_1)U_{\text{in}}(x_0)|0^{\otimes n}\rangle|^2$, that provides the kernel.


In [ ]:
sampler = Sampler()

first_two_inputs = np.concatenate(norm_train_data[:2]).flatten()

job = sampler.run(manual_kernel, parameter_values=first_two_inputs, shots=10000)

# quasi_dists[0] is the probability distribution expected from measured counts of the manual_kernel
fidelity = job.result().quasi_dists[0].get(0, 0.)
print(f'|<φ(x_0)|φ(x_1)>|^2 = {fidelity}')

You can do the same thing using FidelityQuantumKernel class.

In [ ]:
# FidelityQuantumKernel creates a Sampler instance internally
q_kernel = FidelityQuantumKernel(feature_map=feature_map)

bind_params = dict(zip(feature_map.parameters, norm_train_data[0]))
feature_map_0 = feature_map.assign_parameters(bind_params)
bind_params = dict(zip(feature_map.parameters, norm_train_data[1]))
feature_map_1 = feature_map.assign_parameters(bind_params)

qc_circuit = q_kernel.fidelity.create_fidelity_circuit(feature_map_0, feature_map_1)
qc_circuit.decompose().decompose().draw('mpl')

FidelityQuantumKernel allows you to visualize the kernel matrix. Here we make the plots of kernel matrix calculated only from the training data, and that from the training and test data.


In [ ]:
matrix_train = q_kernel.evaluate(x_vec=norm_train_data)
matrix_test = q_kernel.evaluate(x_vec=norm_test_data, y_vec=norm_train_data)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train), interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_test), interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("validation kernel matrix")
plt.show()

Finally, the data are classified into signal and background using support vector machine implemented in sklearn package.

In [ ]:
qc_svc = SVC(kernel='precomputed') # Default value of hyperparameter (C) is 1
qc_svc.fit(matrix_train, train_label)

train_score = qc_svc.score(matrix_train, train_label)
test_score = qc_svc.score(matrix_test, test_label)

print(f'Precomputed kernel: Classification Train score: {train_score*100}%')
print(f'Precomputed kernel: Classification Test score:  {test_score*100}%')

# Reconstruction of Charged Particles (Tracking)

First, let us import necessary modules.

In [ ]:
import os
import sys
import logging

repo_dir = os.path.join(os.environ['HOME'], 'kmi-school-2024')
sys.path.append(repo_dir)

if not os.path.exists(os.path.join(repo_dir, 'qc_workbook', 'hepqpr')):
    import subprocess

    proc = subprocess.Popen(['git', 'submodule', 'init'], cwd=repo_dir)
    proc.wait()

    proc = subprocess.Popen(['git', 'submodule', 'update'], cwd=repo_dir)
    proc.wait()

    #os.symlink(os.path.join(repo_dir, 'hepqpr-qallse', 'src', 'hepqpr'), os.path.join(repo_dir, 'qc_workbook', 'hepqpr'))

    %pip install 'git+https://github.com/LAL/trackml-library.git'
    %pip install dwave-qbsolv
    %pip install dwave-neal

In [ ]:
def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray or dict): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    elif isinstance(state_vector, Statevector):
        binary_string = list(state_vector.sample().keys())[0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x

In [ ]:
from hepqpr.qallse.dsmaker import create_dataset

density = 0.0015
output_path = os.getcwd()+'/ds'
prefix = 'ds'+str(int(1000*density))

metadata, path = create_dataset(
    density=density,
    output_path=output_path,
    prefix=prefix,
    gen_doublets=True
)

In [ ]:
from hepqpr.qallse import *

# ==== BUILD CONFIG
loglevel = logging.INFO

input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
output_path = os.getcwd()+'/ds/'+prefix+'/'

model_class = QallseD0  # model class to use
extra_config = dict()  # model config

dump_config = dict(
    output_path = os.getcwd()+'/ds/'+prefix+'/',
    prefix=prefix+'_',
    xplets_kwargs=dict(format='json', indent=3), # use json (vs "pickle") and indent the output
    qubo_kwargs=dict(w_marker=None, c_marker=None) # save the real coefficients VS generic placeholders
)

# ==== configure logging
logging.basicConfig(
    stream=sys.stderr,
    format="%(asctime)s.%(msecs)03d [%(name)-15s %(levelname)-5s] %(message)s",
    datefmt='%Y-%m-%dT%H:%M:%S')

logging.getLogger('hepqpr').setLevel(loglevel)

# ==== build model
# load data
dw = DataWrapper.from_path(input_path)
doublets = pd.read_csv(input_path.replace('-hits.csv', '-doublets.csv'))

# build model
model = model_class(dw, **extra_config)
model.build_model(doublets)

# dump model to a file
dumper.dump_model(model, **dump_config)

In [ ]:
import pickle
from os.path import join as path_join

from hepqpr.qallse.other.stdout_redirect import capture_stdout
from hepqpr.qallse.other.dw_timing_recorder import solver_with_timing, TimingRecord
from hepqpr.qallse.plotting import *


# ==== RUN CONFIG
nreads = 10
nseed = 1000000

loglevel = logging.INFO

input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
qubo_path = os.getcwd()+'/ds/'+prefix+'/'

# ==== configure logging
logging.basicConfig(
    stream=sys.stdout,
    format="%(asctime)s.%(msecs)03d [%(name)-15s %(levelname)-5s] %(message)s",
    datefmt='%Y-%m-%dT%H:%M:%S')

logging.getLogger('hepqpr').setLevel(loglevel)

# ==== build model
# load data
dw = DataWrapper.from_path(input_path)
pickle_file = prefix+'_qubo.pickle'
with open(path_join(qubo_path, pickle_file), 'rb') as f:
    Q = pickle.load(f)
#print(Q)

import time
start_time = time.process_time()

# Sample qubo

# --- neal
import neal
sampler = neal.SimulatedAnnealingSampler()
#import dimod
#sampler = dimod.RandomSampler()
response = sampler.sample_qubo(Q, num_reads=nreads, seed=nseed)

exec_time = time.process_time() - start_time
print(f'QUBO of size {len(Q)} sampled in {exec_time:.2f}s (NEAL).')
print('')


# get the results
all_doublets = Qallse.process_sample(next(response.samples()))
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

# compute stats
en0 = dw.compute_energy(Q)
en = response.record.energy[0]
occs = response.record.num_occurrences

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

# print stats
print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

# plotting examples
dims = ['x', 'y']
dout = 'plot_'+prefix+'_tracks_found.html'
iplot_results(dw, final_doublets, ms, dims=dims, filename=dout)
#iplot_results_tracks(dw, final_tracks)

## Hamiltonian Formulation and VQE Implementation

In order to use VQE for optimization, the problem will need to be formulated in the form of Hamiltonian. If the problem is formulated such that the solution corresponds to the lowest energy state of the Hamiltonian, the VQE could solve the problem by finding such state.

### QUBO Format

Under this setup, the next step is whether a given segment is adopted as part of particle tracks or rejected as fake. In a sample of $N$ segments, the adoptation or rejection of $i$-th segment is associated to 1 or 0 of a binary variable $T_i$, and the variable $T_i$ is determined such that the objective function defined as

$$
O(b, T) = \sum_{i=1}^N a_{i} T_i + \sum_{i=1}^N \sum_{j<i}^N b_{ij} T_i T_j
$$

is minimized. Here $a_i$ is the score of $i$-th segment and $b_{ij}$ is the score of the pair of $i$- and $j$-th segments. The objective function becomes smaller by selecting segments that have smaller $a_i$ values (pointing towards the detector center) and are paired with other segments with smaller $b_{ij}$ values (more consistent with a real track) and rejecting otherwise. Once correct segments are identified, the corresponding tracks can be reconstructed with high efficiency. Therefore, solving this minimization problem is the key to tracking.

Let us first read out the scores $a_i$ and $b_{ij}$. The two scores are stored in a two-dimensional array `b`, and `b[i,i]` corresponds to $a_i$.


In [ ]:
n_max = 100

nvar = 0
key_i = []
a_score = np.zeros(n_max)
for (k1, k2), v in Q.items():
    if k1 == k2:
        a_score[nvar] = v
        key_i.append(k1)
        nvar += 1
a_score = a_score[:nvar]

b_score = np.zeros((n_max,n_max))
for (k1, k2), v in Q.items():
    if k1 != k2:
        for i in range(nvar):
            for j in range(nvar):
                if k1 == key_i[i] and k2 == key_i[j]:
                    if i < j:
                        b_score[j][i] = v
                    else:
                        b_score[i][j] = v

b_score = b_score[:nvar,:nvar]

print(f'# of Segments: {nvar}')
# Print out the first 5x5
print(a_score[:5])
print(b_score[:5, :5])

### Ising Format

The QUBO objective function is not the form of Hamiltonian (i.e, not Hermitian operator). Therefore, the objective function needs to be transformed before solving with VQE. Given that $T_i$ takes a binary value $\{0, 1\}$, a new variable $s_i$ with values of $\{+1, -1\}$ can be defined by

$$
T_i = \frac{1}{2} (1 - s_i).
$$

Note that $\{+1, -1\}$ is the eigenvalue of Pauli operator. By replacing $s_i$ with Pauli $Z$ operator acting on $i$-th qubit, the following objective Hamiltonian for which computational basis states in $N$-qubit system correspond to the eigenstates that encode adoptation or rejection of the segments is obtained.

$$
H(h, J, s) = \sum_{i=1}^N h_i Z_i + \sum_{i=1}^N \sum_{j<i}^N J_{ij} Z_i Z_j + \text{(constant)}
$$

The form of this Hamiltonian is the same as Ising model Hamiltonian, which often appears in various fields of natural science. The $\text{constant}$ is constant and has no impact in variational method, hence is ignored in the rest of this exercise.

### Exercise

By following the above prescription, please calculate the coefficients $h_i$ and $J_{ij}$ of the Hamiltonian in the next cell.

In [ ]:
num_qubits = nvar

coeff_h = np.zeros(num_qubits)
coeff_J = np.zeros((num_qubits, num_qubits))

##################
### EDIT BELOW ###
##################

# Calculate coeff_h and coeff_J from a_score and b_score
coeff_h = -(a_score / 2. + (np.sum(b_score, axis=0) + np.sum(b_score, axis=1)) / 4.)
coeff_J = b_score / 4.

##################
### EDIT ABOVE ###
##################

Next, let us define the Hamiltonian used in VQE as a SparsePauliOp object. In {ref}`vqe_imp` the SparsePauliOp was used to define a single Pauli string $ZXY$, but the same class can be used for the sum of Pauli strings. For example,

$$
H = 0.2 IIZ + 0.3 ZZI + 0.1 ZIZ
$$

can be expressed as

```python
H = SparsePauliOp(['IIZ', 'ZZI', 'ZIZ'], coeffs=[0.2, 0.3, 0.1])
```

Note that the qubits are ordered from right to left (the most right operator acts on the 0-th qubit) according to the rule in Qiskit.

### Exercise

Pick up all the Pauli strings with non-zero coefficients and make the array of corresponding coefficients.

In [ ]:
pauli_products = []
coeffs = []

##################
### EDIT BELOW ###
##################

for iq in range(num_qubits):
    if np.isclose(coeff_h[iq], 0.):
        continue

    pauli_products.append(('I' * (num_qubits - iq - 1)) + 'Z' + ('I' * iq))
    coeffs.append(coeff_h[iq])

for iq in range(num_qubits):
    for jq in range(iq):
        if np.isclose(coeff_J[iq, jq], 0.):
            continue

        pauli = 'I' * (num_qubits - iq - 1)
        pauli += 'Z'
        pauli += 'I' * (iq - jq - 1)
        pauli += 'Z'
        pauli += 'I' * jq
        pauli_products.append(pauli)

        coeffs.append(coeff_J[iq, jq])

##################
### EDIT ABOVE ###
##################

hamiltonian = SparsePauliOp(pauli_products, coeffs=coeffs)

### Executing VQE

Now we try to approximately obtain the lowest energy eigenvalues using VQE with the Hamiltonian defined above. But, before doing that, let us diagonalize the Hamiltonian matrix and calculate the exact energy eigenvalues and eigenstates.

In [ ]:
# Diagonalize the Hamiltonian and calculate the energy eigenvalues and eigenstates
ee = NumPyMinimumEigensolver()
result_diag = ee.compute_minimum_eigenvalue(hamiltonian)

# Print out the combination of qubits corresponding to the lowest energy
print(f'Minimum eigenvalue (diagonalization): {result_diag.eigenvalue.real}')
# Expand the state with computational bases and select the one with the highest probability
optimal_segments_diag = OptimizationApplication.sample_most_likely(result_diag.eigenstate)
print(f'Optimal segments (diagonalization): {optimal_segments_diag}')

In [ ]:
backend = AerSimulator()
# Create Estimator instance
estimator = BackendEstimator(backend)

# Define variational form of VQE using a built-in function called TwoLocal.
ansatz = TwoLocal(num_qubits, 'ry', 'cz', 'linear', reps=1)

# Optimizer
optimizer_name = 'SPSA'

if optimizer_name == 'SPSA':
    optimizer = SPSA(maxiter=300)
    grad = ParamShiftEstimatorGradient(estimator)

elif optimizer_name == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
    grad = None

# Initialize parameters with random values
rng = np.random.default_rng()
init = rng.uniform(0., 2. * np.pi, size=len(ansatz.parameters))

In [ ]:
# Make VQE object and search for the ground state
vqe = VQE(estimator, ansatz, optimizer, gradient=grad, initial_point=init)
result_vqe = vqe.compute_minimum_eigenvalue(hamiltonian)

# Create state vector from the ansatz using optimized parameters
optimal_state = Statevector(ansatz.assign_parameters(result_vqe.optimal_parameters))

# Print out the combination of qubits with the lowest energy
print(f'Minimum eigenvalue (VQE): {result_vqe.eigenvalue.real}')
optimal_segments_vqe = OptimizationApplication.sample_most_likely(optimal_state)
print(f'Optimal segments (VQE): {optimal_segments_vqe}')

### Visual Inspection

Even if the tracking works successfully, expressing the answer as a string of 0s and 1s is a bit dry. Run the following code to visually confirm if correct tracks are found.

This code viualizes the detector hits used in QUBO by projecting them onto a plane perpendicular to the beam axis and shows which detector hits are selected after optimization. The green lines correspond to found tracks and the blue lines, altogether with the green ones, correspond to all track candidates. In this exercise, only small number of qubits are used and therefore most of tracks are not found. However, it shows that a correct track is successfully found from the presence of green line.

In [ ]:
from hepqpr.qallse import DataWrapper, Qallse, TrackRecreaterD
from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
from hepqpr.qallse.utils import diff_rows

#optimal_segments = optimal_segments_vqe
optimal_segments = optimal_segments_diag

# Since each segment has ID, the data is passed to Qallse in the form of {ID: 0 or 1}
#with h5py.File('data/QUBO_05pct_input.h5', 'r') as source:
#   triplet_keys = map(lambda key: key.decode('UTF-8'), source['triplet_keys'][()])
#
#samples = dict(zip(triplet_keys, optimal_segments))
samples = dict(zip(key_i, optimal_segments))

# get the results
all_doublets = Qallse.process_sample(samples)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

#dw = DataWrapper.from_path('data/event000001000-hits.csv')
input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
dw = DataWrapper.from_path(input_path)

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)